# Soft-voting and export final submission for DRAC2022

* creator: Jungrae Cho (team: FindDR)
* created: November 25 2022

5 models trained with 20eps should be prepared.

* resnet50d
* efficientnet_b3a
* efficientnet_b3
* skresnext50_32x4d
* seresnext50d_32x4d

Every model should export its own `submission.csv` like:
```
['timm-efficient_b3a-drac2022-task3-20220911-1_FindDR.csv',
 'timm-seresnext50_32x4d-drac2022-task3-20220911-1_FindDR.csv',
 'timm-efficient_b3-drac2022-task3-20220911-1_FindDR.csv',
 'timm-resnet50d-drac2022-task3-20220911-1_FindDR.csv',
 'timm-skresnext50_32x4d-drac2022-task3-20220911-1_FindDR.csv']
1
```

In [ ]:
import pandas as pd
import numpy as np
from glob import glob

In [ ]:
def softmax_np(arr):
    return np.exp(arr)/np.sum(np.exp(arr))

In [ ]:
csv_paths = glob("*.csv")
csv_paths

`csv_paths` should be like this:
```
['timm-efficient_b3a-drac2022-task3-20220911-1_FindDR.csv',
 'timm-seresnext50_32x4d-drac2022-task3-20220911-1_FindDR.csv',
 'timm-efficient_b3-drac2022-task3-20220911-1_FindDR.csv',
 'timm-resnet50d-drac2022-task3-20220911-1_FindDR.csv',
 'timm-skresnext50_32x4d-drac2022-task3-20220911-1_FindDR.csv']
 ```

In [ ]:
dfs = []
for csv_path in csv_paths:
    dfs.append(pd.read_csv(csv_path))

In [ ]:
cases = dfs[0]["case"].values
cls = []
P0 = []
P1 = []
P2 = []
for df in dfs:
    cls.append(df["class"].values)
    P0.append(df["P0"].values)
    P1.append(df["P1"].values)
    P2.append(df["P2"].values)

In [ ]:
cls = np.array(cls, np.uint8)
P0 = np.array(P0, np.float32)
P1 = np.array(P1, np.float32)
P2 = np.array(P2, np.float32)

In [ ]:
cls.shape, P0.shape, P1.shape, P2.shape

In [ ]:
hard_voting = np.round(np.mean(cls, axis=0)).astype(np.uint8)
P0 = np.mean(P0, axis=0)
P1 = np.mean(P1, axis=0)
P2 = np.mean(P2, axis=0)

In [ ]:
soft_voting = np.array([P0,P1,P2]).T

In [ ]:
soft_cls = []
soft_P0 = []
soft_P1 = []
soft_P2 = []
for i in range(soft_voting.shape[0]):
    prob = soft_voting[i]
    prob = softmax_np(prob)
    soft_cls.append(np.argmax(prob))
    soft_P0.append(prob[0])
    soft_P1.append(prob[1])
    soft_P2.append(prob[2])
    print(np.sum(prob))
soft_cls = np.array(soft_cls, np.uint8)

In [ ]:
hard_voting

In [ ]:
soft_cls

In [ ]:
submission_dict = {
    "case": cases,
    "class": soft_cls,
    "P0": soft_P0,
    "P1": soft_P1,
    "P2": soft_P2,
}

In [ ]:
submission_df = pd.DataFrame(submission_dict, columns=['case', 'class', 'P0', 'P1', 'P2'])

submission_df.to_csv(f"Task3_soft_voting_TEAMNAME.csv", index=False)

In [ ]:
submission_df